# S02: データ変換（HTML → Parquet）

HTMLをパースしてParquet形式で保存

**実行環境**: Sagemaker

---
## 1. 環境設定

In [ ]:
import sys
import os
from pathlib import Path
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# プロジェクトルート
PROJECT_ROOT = Path('../../')
sys.path.insert(0, str(PROJECT_ROOT / 'src'))

# データパス
HTML_DIR = PROJECT_ROOT / 'data' / 'raceHTML'
PROCESSED_DIR = PROJECT_ROOT / 'data' / 'processed'
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

# ファイルパス
RACES_PARQUET = PROCESSED_DIR / 'races.parquet'
RESULTS_PARQUET = PROCESSED_DIR / 'results.parquet'

print(f"HTML_DIR: {HTML_DIR.resolve()}")
print(f"出力先: {PROCESSED_DIR.resolve()}")

In [ ]:
from scraper.parser import parse_multiple_html_full
print("✅ モジュール読み込み完了")

---
## 2. パース設定

**開発中**: 各年100件ずつで確認  
**本番**: `LIMIT_PER_YEAR = None` で全件処理

In [ ]:
# ========== 設定 ==========

# 開発モード: 各年100件に制限
LIMIT_PER_YEAR = 100

# 本番モード: 全件処理（下のコメントを外す）
# LIMIT_PER_YEAR = None

print(f"処理モード: {'開発（各年{LIMIT_PER_YEAR}件）' if LIMIT_PER_YEAR else '本番（全件）'}")

---
## 3. HTMLパース実行

In [ ]:
def progress_callback(processed, total, skipped=0):
    pct = processed / total * 100 if total > 0 else 0
    print(f"\r進捗: {processed:,}/{total:,} ({pct:.1f}%)", end='')

print("📊 HTMLパースを開始...")
if LIMIT_PER_YEAR:
    print(f"   ⚠️ 開発モード: 各年 {LIMIT_PER_YEAR} 件まで")

race_list, horse_list = parse_multiple_html_full(
    HTML_DIR,
    years=None,
    progress_callback=progress_callback,
    limit_per_year=LIMIT_PER_YEAR
)

print(f"\n\n✅ パース完了")
print(f"   レース数: {len(race_list):,}")
print(f"   出走馬データ数: {len(horse_list):,}")

In [ ]:
# DataFrameに変換
races_df = pd.DataFrame(race_list)
results_df = pd.DataFrame(horse_list)

print(f"races_df: {races_df.shape}")
print(f"results_df: {results_df.shape}")

In [ ]:
# Parquet形式で保存
races_df.to_parquet(RACES_PARQUET, index=False, compression='snappy')
results_df.to_parquet(RESULTS_PARQUET, index=False, compression='snappy')

print("✅ Parquet保存完了")
print(f"   races.parquet: {RACES_PARQUET.stat().st_size / 1024 / 1024:.2f} MB")
print(f"   results.parquet: {RESULTS_PARQUET.stat().st_size / 1024 / 1024:.2f} MB")

---
## 4. データ確認

In [ ]:
print("=== データ統計 ===")
print(f"レース数: {len(races_df):,}")
print(f"出走馬データ数: {len(results_df):,}")

if 'year' in races_df.columns:
    print("\n=== 年別レース数 ===")
    print(races_df.groupby('year').size().to_string())

In [ ]:
# サンプル表示
print("=== レースデータ サンプル ===")
display(races_df.tail(5))

In [ ]:
print("=== 出走馬データ サンプル ===")
display(results_df.tail(5))

---
## 5. GitHubにプッシュ

```bash
git add data/processed/*.parquet
git commit -m "Update race data parquet files"
git push
```

---
## 6. オンラインからの読み込み

```python
import pandas as pd

# GitHub RAW URLから読み込み
BASE = "https://raw.githubusercontent.com/iinumac/keiba_prediction/main/data/processed"

races_df = pd.read_parquet(f"{BASE}/races.parquet")
results_df = pd.read_parquet(f"{BASE}/results.parquet")

# 特定の馬を検索
horse_results = results_df[results_df['horse_id'] == "2022110077"]
```